# Práctica Clase 6

1) Cargar el dataset "Emisiones_CO2.csv" provisto en la clase 2 en un Dataframe de Pandas, quitar los registros que contengan valores faltantes e implementar una nueva columna, que contenga el resultado de una función Hash aplicada sobre el campo "Código de País" y se denomine "Clave_Hash".<br>
Consideraciones: Se puede utilizar la función provista.


In [3]:
def hash_function(key):
    return sum(index * ord(character) for index, character in enumerate(repr(key), start=1))

Voy a empezar entendiendo qué hace la función hash_function. Lo primero que puedo notar es que repr() es la representtación de un objeto y la representación de key va a incluir las comillas. Ejemplo: len(repr('pato')) va a ser 6, mientras que len(str('pato')) va a ser 4.

In [4]:
len(str('pato'))

4

In [5]:
len(repr('pato'))

6

(index * ord(character) for index, character in enumerate(repr(key), start=1)) es un generador. Lo probamos de la siguiente manera:


In [6]:
print((index * ord(character) for index, character in enumerate(repr('a'), start=1)))

<generator object <genexpr> at 0x000001ADAB0366B0>


Vamos a pensar qué genera este generador:
Es una tuple comprehension que multiplica el índice por el valor numérico del caracter en cada iteración, empezando por 1. Vamos a ver ejemplos de valores numéricos y de cómo se implementa la función.

In [7]:
repr('a')

"'a'"

Vemos que la representación de 'a' consta de 3 caracteres (', a y ').

In [8]:
ord("'") # Está dos veces, en la primera y en la tercera posición.

39

In [9]:
ord('a')

97

Sabemos que enumerate nos va a dar un índice y un valor. En este caso, el índice con el que empezamos no va a ser 0, va a ser 1 (start=1). Por lo tanto, para cada caracter nos va a dar la multiplicación entre el índice y el valor numérico del caracter. En el caso de 'a':

': índice = 1; ord = 39 ==> 1 * 39 = 39
a: índice = 2; ord = 97 ==> 2 * 97 = 194
': índice = 3; ord = 39 ==> 3 * 39 = 117

Para ver el contenido del generador y comprobar si funciona así, lo podemos guardar, por ejemplo, en una tupla:

In [10]:
a = tuple((index * ord(character) for index, character in enumerate(repr('a'), start=1)))
print(a)


(39, 194, 117)


Efectivamente, funciona así. Solo falta sum(), que suma todos los valores de la tupla. En el caso de 'a' debería devolver 350.
Probemos la función con 'a':

In [11]:
hash_function('a')


350

Ahora, antes de probar la función, hagamos los cálculos para 'pato' y 'tapo', para comprobar si estamos en lo correcto:

In [12]:
print(repr('pato'))
print(ord("'"))
print(ord('p'))
print(ord('a'))
print(ord('t'))
print(ord('o'))
print(ord("'"))



'pato'
39
112
97
116
111
39


Siguiendo el mismo procedimiento que antes:
1 * 39 ==> 39;
2 * 112 ==> 224;
3 * 97 ==> 291;
4 * 116 ==> 464;
5 * 111 ==> 555;
6 * 39 ==> 234;

In [13]:
a= (39, 224, 291, 464, 555, 234)
sum(a)

1807

Y ahora usamos la función:


In [14]:
hash_function('pato')

1807

Comprobemos el último ejemplo:


In [15]:
print(repr('tapo'))
print(ord("'"))
print(ord('t'))
print(ord('a'))
print(ord('p'))
print(ord('o'))
print(ord("'"))

'tapo'
39
116
97
112
111
39


1 * 39 ==> 39;
2 * 116 ==> 232;
3 * 97 ==> 291;
4 * 112 ==> 448;
5 * 111 ==> 555;
6 * 39 ==> 234;

In [16]:
a = (39, 232, 291, 448, 555, 234)
sum(a)

1799

In [17]:
print(hash_function('tapo'))

1799


In [18]:
print(hash_function('pato'))
print(hash_function('tapo'))

1807
1799


En el homework resuelto hay una explicación errónea, si empezamos por 0 no da todo 0, solo el primer valor. Y, como despues vamos sumando los otros (que no tienen índice 0 y van a devolver el resultado de una multiplicación), el resultado final no va a ser 0. Lo muestro en código:

In [19]:
def hash_function_1(key):
    return sum(index * ord(character) for index, character in enumerate(repr(key)))

print(hash_function_1('pato'))
print(hash_function_1('tapo'))

1293
1285


In [9]:
import pandas as pd

Primero, creamos el df:

In [21]:
df = pd.read_csv('Emisiones_CO2.csv',sep='|',decimal=',', encoding='latin-1')

Borramos los registros que contengan valores faltantes y guardamos el df:

In [22]:
df.dropna(inplace=True)

Creamos la nueva columna:

In [23]:
df['Clave_Hash'] = 0

Reemplazamos los valores por el resultado de aplicar la función hash_function al campo 'Código de país':

In [24]:
df['Clave_Hash'] = df['Código de país'].apply(hash_function)

In [25]:
df

,Código de país,Nombre del país,Región,Año,CO2 (kt),CO2 per cápita (toneladas métricas),Clave_Hash
26,ABW,Aruba,América Latina y Caribe,1986,"179,683",2.868319,910
27,ABW,Aruba,América Latina y Caribe,1987,"447,374",7.234964,910
28,ABW,Aruba,América Latina y Caribe,1988,"612,389",10.026508,910
29,ABW,Aruba,América Latina y Caribe,1989,"649,059",10.634733,910
30,ABW,Aruba,América Latina y Caribe,1990,"1.840,83",29.620165,910
...,...,...,...,...,...,...,...
11122,ZWE,Zimbabue,África subsahariana,2007,"9.732,22",0.731867,951
11123,ZWE,Zimbabue,África subsahariana,2008,"7.682,37",0.569255,951
11124,ZWE,Zimbabue,África subsahariana,2009,"8.239,75",0.600521,951
11125,ZWE,Zimbabue,África subsahariana,2010,"9.028,15",0.646073,951


Es posible que se repitan valores de has, debido a la suma de la combinacion de índices y valores en distintas posiciones. Pero el ejercicio no habla de esto.

Voy a hacer lo mismo, pero con otra función hash, para que no de valores repetidos.

In [7]:
def mi_hash_function(key):
    hash = ''
    key = str(key)
    for letra in key:
        hash += str(ord(letra) + ord(letra) * 3)
    for letra in key:
        hash += str(hex(ord(letra)))
    return hash

print(mi_hash_function('pato'))
print(mi_hash_function('Pato'))
print(mi_hash_function('a'))
        


4483884644440x700x610x740x6f
3203884644440x500x610x740x6f
3880x61


In [16]:
df = pd.read_csv('Emisiones_CO2.csv',sep='|',decimal=',', encoding='latin-1')

In [17]:
df.dropna(inplace=True)

In [18]:
df['Clave_Hash'] = df['Código de país'].apply(mi_hash_function)

In [19]:
df

,Código de país,Nombre del país,Región,Año,CO2 (kt),CO2 per cápita (toneladas métricas),Clave_Hash
26,ABW,Aruba,América Latina y Caribe,1986,"179,683",2.868319,2602643480x410x420x57
27,ABW,Aruba,América Latina y Caribe,1987,"447,374",7.234964,2602643480x410x420x57
28,ABW,Aruba,América Latina y Caribe,1988,"612,389",10.026508,2602643480x410x420x57
29,ABW,Aruba,América Latina y Caribe,1989,"649,059",10.634733,2602643480x410x420x57
30,ABW,Aruba,América Latina y Caribe,1990,"1.840,83",29.620165,2602643480x410x420x57
...,...,...,...,...,...,...,...
11122,ZWE,Zimbabue,África subsahariana,2007,"9.732,22",0.731867,3603482760x5a0x570x45
11123,ZWE,Zimbabue,África subsahariana,2008,"7.682,37",0.569255,3603482760x5a0x570x45
11124,ZWE,Zimbabue,África subsahariana,2009,"8.239,75",0.600521,3603482760x5a0x570x45
11125,ZWE,Zimbabue,África subsahariana,2010,"9.028,15",0.646073,3603482760x5a0x570x45


2. A partir del Dataframe creado en el punto 1, construir uno nuevo, que contenga solo los valores distintos de la tupla "Clave_Hash", "Código de País" , "Nombre de país" y "Región". Quitando luego del dataframe original los campos "Código de País" , "Nombre de país" y "Región"

Creamos el nuevo df.

In [20]:
df_nuevo = df.copy()

In [21]:
df_nuevo

,Código de país,Nombre del país,Región,Año,CO2 (kt),CO2 per cápita (toneladas métricas),Clave_Hash
26,ABW,Aruba,América Latina y Caribe,1986,"179,683",2.868319,2602643480x410x420x57
27,ABW,Aruba,América Latina y Caribe,1987,"447,374",7.234964,2602643480x410x420x57
28,ABW,Aruba,América Latina y Caribe,1988,"612,389",10.026508,2602643480x410x420x57
29,ABW,Aruba,América Latina y Caribe,1989,"649,059",10.634733,2602643480x410x420x57
30,ABW,Aruba,América Latina y Caribe,1990,"1.840,83",29.620165,2602643480x410x420x57
...,...,...,...,...,...,...,...
11122,ZWE,Zimbabue,África subsahariana,2007,"9.732,22",0.731867,3603482760x5a0x570x45
11123,ZWE,Zimbabue,África subsahariana,2008,"7.682,37",0.569255,3603482760x5a0x570x45
11124,ZWE,Zimbabue,África subsahariana,2009,"8.239,75",0.600521,3603482760x5a0x570x45
11125,ZWE,Zimbabue,África subsahariana,2010,"9.028,15",0.646073,3603482760x5a0x570x45


Ahora vamos a sacar los valores únicos que pide la consigna:

In [29]:
df_nuevo[['Clave_Hash', 'Código de país','Nombre del país', 'Región']]

,Clave_Hash,Código de país,Nombre del país,Región
26,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
27,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
28,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
29,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
30,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
...,...,...,...,...
11122,3603482760x5a0x570x45,ZWE,Zimbabue,África subsahariana
11123,3603482760x5a0x570x45,ZWE,Zimbabue,África subsahariana
11124,3603482760x5a0x570x45,ZWE,Zimbabue,África subsahariana
11125,3603482760x5a0x570x45,ZWE,Zimbabue,África subsahariana


Vemos que hay 9029 registros.

In [30]:
df_nuevo[['Clave_Hash', 'Código de país','Nombre del país', 'Región']].drop_duplicates()

,Clave_Hash,Código de país,Nombre del país,Región
26,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
87,2603122720x410x4e0x44,AND,Andorra,Europa y Asia Central
104,2602802840x410x460x47,AFG,Afganistán,Sur de Asia
156,2602843160x410x470x4f,AGO,Angola,África subsahariana
208,2603042640x410x4c0x42,ALB,Albania,Europa y Asia Central
...,...,...,...,...
10867,3562763080x590x450x4d,YEM,Yemen,Oriente Medio y Norte de África
10919,3602602800x5a0x410x46,ZAF,Sudáfrica,África subsahariana
10971,2683162720x430x4f0x44,COD,Congo (República Democrática),África subsahariana
11027,3603082640x5a0x4d0x42,ZMB,Zambia,África subsahariana


Y luego de dejar los valores únicos, 199. Para verificarlo, voy a crear un .csv.

In [31]:
df_prueba = df_nuevo[['Clave_Hash', 'Código de país','Nombre del país', 'Región']].drop_duplicates()

In [33]:
df_prueba.to_csv('pruebaHomework.csv', sep=',')

Perfecto. Acá solo dejamos los campos especificados sin duplicados. Ya que todo está correcto, creo el nuevo df.

In [71]:
df_paises = df_prueba.copy()

In [72]:
df_paises

,Clave_Hash,Código de país,Nombre del país,Región
26,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
87,2603122720x410x4e0x44,AND,Andorra,Europa y Asia Central
104,2602802840x410x460x47,AFG,Afganistán,Sur de Asia
156,2602843160x410x470x4f,AGO,Angola,África subsahariana
208,2603042640x410x4c0x42,ALB,Albania,Europa y Asia Central
...,...,...,...,...
10867,3562763080x590x450x4d,YEM,Yemen,Oriente Medio y Norte de África
10919,3602602800x5a0x410x46,ZAF,Sudáfrica,África subsahariana
10971,2683162720x430x4f0x44,COD,Congo (República Democrática),África subsahariana
11027,3603082640x5a0x4d0x42,ZMB,Zambia,África subsahariana


Para que quede más prolijo, reseteo los índices.

In [73]:
df_paises.reset_index(drop=True, inplace=True)

Y para que empiece en 1 y no en 0:


In [74]:
df_paises.index = df_paises.index + 1

In [75]:
df_paises

,Clave_Hash,Código de país,Nombre del país,Región
1,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
2,2603122720x410x4e0x44,AND,Andorra,Europa y Asia Central
3,2602802840x410x460x47,AFG,Afganistán,Sur de Asia
4,2602843160x410x470x4f,AGO,Angola,África subsahariana
5,2603042640x410x4c0x42,ALB,Albania,Europa y Asia Central
...,...,...,...,...
195,3562763080x590x450x4d,YEM,Yemen,Oriente Medio y Norte de África
196,3602602800x5a0x410x46,ZAF,Sudáfrica,África subsahariana
197,2683162720x430x4f0x44,COD,Congo (República Democrática),África subsahariana
198,3603082640x5a0x4d0x42,ZMB,Zambia,África subsahariana


Ahora vamos a sacar del df original (df_nuevo) los campos "Código de país" , "Nombre del país" y "Región"

In [77]:
df_nuevo.drop(['Código de país', 'Nombre del país', 'Región'], axis=1)

,Año,CO2 (kt),CO2 per cápita (toneladas métricas),Clave_Hash
26,1986,"179,683",2.868319,2602643480x410x420x57
27,1987,"447,374",7.234964,2602643480x410x420x57
28,1988,"612,389",10.026508,2602643480x410x420x57
29,1989,"649,059",10.634733,2602643480x410x420x57
30,1990,"1.840,83",29.620165,2602643480x410x420x57
...,...,...,...,...
11122,2007,"9.732,22",0.731867,3603482760x5a0x570x45
11123,2008,"7.682,37",0.569255,3603482760x5a0x570x45
11124,2009,"8.239,75",0.600521,3603482760x5a0x570x45
11125,2010,"9.028,15",0.646073,3603482760x5a0x570x45


Está bien, por lo que vamos a guardarlo en otro df. Estoy guardando todo en distintos df para mantener los originales, pero no por lo que pide el ejercicio en sí. Se podrían modificar directamente. Solo para que quede más claro y veamos con qué campos trabajamos en cada momento.

In [78]:
df_datos = df_nuevo.drop(['Código de país', 'Nombre del país', 'Región'], axis=1)

In [79]:
df_datos

,Año,CO2 (kt),CO2 per cápita (toneladas métricas),Clave_Hash
26,1986,"179,683",2.868319,2602643480x410x420x57
27,1987,"447,374",7.234964,2602643480x410x420x57
28,1988,"612,389",10.026508,2602643480x410x420x57
29,1989,"649,059",10.634733,2602643480x410x420x57
30,1990,"1.840,83",29.620165,2602643480x410x420x57
...,...,...,...,...
11122,2007,"9.732,22",0.731867,3603482760x5a0x570x45
11123,2008,"7.682,37",0.569255,3603482760x5a0x570x45
11124,2009,"8.239,75",0.600521,3603482760x5a0x570x45
11125,2010,"9.028,15",0.646073,3603482760x5a0x570x45


Ahora solo falta unir ambas tablas por el campo común ("Clave_Hasf"):


In [80]:
df_datos.merge(df_paises, on=['Clave_Hash'])

,Año,CO2 (kt),CO2 per cápita (toneladas métricas),Clave_Hash,Código de país,Nombre del país,Región
0,1986,"179,683",2.868319,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
1,1987,"447,374",7.234964,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
2,1988,"612,389",10.026508,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
3,1989,"649,059",10.634733,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
4,1990,"1.840,83",29.620165,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
...,...,...,...,...,...,...,...
9024,2007,"9.732,22",0.731867,3603482760x5a0x570x45,ZWE,Zimbabue,África subsahariana
9025,2008,"7.682,37",0.569255,3603482760x5a0x570x45,ZWE,Zimbabue,África subsahariana
9026,2009,"8.239,75",0.600521,3603482760x5a0x570x45,ZWE,Zimbabue,África subsahariana
9027,2010,"9.028,15",0.646073,3603482760x5a0x570x45,ZWE,Zimbabue,África subsahariana


Vemos que tenemos nuevamente los 9029 registros y que está todo bien. Ahora voy a crear un nuevo df con esta unión.

In [81]:
df_final = df_datos.merge(df_paises, on=['Clave_Hash'])

In [82]:
df_final

,Año,CO2 (kt),CO2 per cápita (toneladas métricas),Clave_Hash,Código de país,Nombre del país,Región
0,1986,"179,683",2.868319,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
1,1987,"447,374",7.234964,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
2,1988,"612,389",10.026508,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
3,1989,"649,059",10.634733,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
4,1990,"1.840,83",29.620165,2602643480x410x420x57,ABW,Aruba,América Latina y Caribe
...,...,...,...,...,...,...,...
9024,2007,"9.732,22",0.731867,3603482760x5a0x570x45,ZWE,Zimbabue,África subsahariana
9025,2008,"7.682,37",0.569255,3603482760x5a0x570x45,ZWE,Zimbabue,África subsahariana
9026,2009,"8.239,75",0.600521,3603482760x5a0x570x45,ZWE,Zimbabue,África subsahariana
9027,2010,"9.028,15",0.646073,3603482760x5a0x570x45,ZWE,Zimbabue,África subsahariana
